In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
directory = './data/track_1/'

# гидро
hydro_1day = pd.read_csv(directory + 'hydro_1day.csv', parse_dates=['date'])
# hydro_coord = pd.read_csv(directory + 'hydro_coord.csv')

# метео — обратите внимание на различие во временной детализации
# meteo_3hours = pd.read_csv(directory + 'meteo_3hours.csv')
# meteo_1day = pd.read_csv(directory + 'meteo_1day.csv')
# meteo_1month = pd.read_csv(directory + 'meteo_1month.csv')
# meteo_coord = pd.read_csv(directory + 'meteo_coord.csv')

# справочники
# reference_water_codes = pd.read_csv(directory + 'reference_water_codes.csv')
# reference_horiz_visib = pd.read_csv(directory + 'reference_horiz_visib.csv')

# тест и трейн
train_df = pd.read_csv(directory + 'train.csv')
test_df = pd.read_csv(directory + 'test.csv')

# Обработка

In [3]:
hydro_1day = pd.read_csv(directory + 'hydro_1day.csv', parse_dates=['date'])



# разделяем код режимной группы
water_code = pd.DataFrame(hydro_1day['water_code'].str.split(',', expand=True).fillna(0).astype(int))
water_code.columns=[f'water_code_{i}' for i in range(5)]


# добавляем в общий датасет
hydro_1day = pd.concat([hydro_1day, water_code], axis=1)
hydro_1day = hydro_1day.drop(['water_code'], 1)
hydro_1day = hydro_1day.sort_values(by=['year', 'day'])
hydro_1day = hydro_1day.reset_index(drop=True)


# удаляем не нужные фичи
hydro_1day = hydro_1day.drop(['month'], 1)

# запонняем пустые значения
hydro_1day[['place', 'snow_height']] = hydro_1day[['place','snow_height']].fillna(0)
hydro_1day.loc[hydro_1day['temp'].isna(), 'temp'] = hydro_1day['temp'].median()

# нормализируем
hydro_1day['place'] = hydro_1day['place'].astype(int)
lb_place = LabelEncoder()
hydro_1day['place'] = lb_place.fit_transform(hydro_1day['place'])
hydro_1day[['stage_avg', 'stage_min', 'stage_max']] = hydro_1day[['stage_avg', 'stage_min', 'stage_max']].abs()


# интерполируем данные для каждой станции
columns_interpolate = ['stage_avg', 'stage_min', 'stage_max', 'temp', 'discharge', 'ice_thickness']
stations = hydro_1day['station_id'].unique()
for i in tqdm(stations):
    for col in columns_interpolate:
        hydro_1day.loc[hydro_1day['station_id'] == i, col] = hydro_1day.loc[hydro_1day['station_id'] == i, col].interpolate(limit_direction='both')
        
        
hydro_1day['discharge'] = hydro_1day['discharge'].fillna(hydro_1day['discharge'].median())

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 37.27it/s]


In [4]:
hydro_1day['real'] = 1
empty_rows = []
stations = hydro_1day['station_id'].unique()

for index, i in tqdm(hydro_1day.groupby(['year', 'day'])):
    for station in stations:
        if station not in i['station_id'].values:     
            row = i.iloc[0].copy()
            row['station_id'] = station
            empty_rows.append(row.values)
            
       
empty_df = pd.DataFrame(empty_rows, columns=hydro_1day.columns)
empty_df.iloc[:,4:] = 0
hydro_1day = pd.concat([hydro_1day, empty_df], ignore_index=True).sort_values(by=['year', 'day'])
# hydro_1day = hydro_1day.append(empty_df)

100%|████████████████████████████████████████████████████████████████████████████| 8960/8960 [00:04<00:00, 1848.48it/s]


In [91]:
hydro_1day.to_csv(directory + 'hydro_1day_prep.csv', index=None)

In [73]:
num_days_data = 14
def get_data(year):
    if year-1 in hydro_1day['year'].values:
        part1 = hydro_1day[(hydro_1day['year']==year-1)].iloc[-num_days_data*len(stations):]    
    else:
        part1 = hydro_1day[(hydro_1day['year']==year)].iloc[-num_days_data*len(stations):]  
        if (year-1)%4:
            days = 365
        else:
            days = 366

        part1['date'] = part1['date'] - pd.Timedelta(days=days)

    part2 = hydro_1day[(hydro_1day['year']==year)].iloc[:num_days_data*len(stations)]
    result = pd.concat([part1, part2])
    result = result.drop(['date'], 1)
    return result

In [74]:
rfc = RandomForestClassifier(n_jobs=-1, random_state=42)

def train_model(df):
    days_X = []
    days_Y = []
    for index1, group in tqdm(df.groupby(['station_id','year'])):

        data = get_data(year=index1[1])

        for i, (index, row) in enumerate(group.iterrows()):
            period = data.iloc[i:num_days_data*len(stations)+i]
            period = period.values.flatten()

            days_X.append(np.append(row.drop('ice_jam').values, period))
            days_Y.append(row['ice_jam'])

    rfc.fit(days_X, days_Y)

In [75]:
def predict(df):
    Y_pred = []
    full_train_df = df.copy()
    for index1, group in tqdm(full_train_df.groupby(['station_id','year'])):

        data = get_data(year=index1[1])
        
        X_days = []

        for i, (index, row) in enumerate(group.iterrows()):
            period = data.iloc[i:num_days_data*len(stations)+i]
            period = period.values.flatten()

            full_train_df.loc[index, 'ice_jam_pred'] = rfc.predict([np.append(row.drop('ice_jam').values, period)])
            
            
    return full_train_df

In [76]:
X_train, X_test = train_test_split(train_df, test_size=0.33, random_state=42, stratify=train_df['ice_jam'])

In [77]:
train_model(X_train)
pred = predict(X_test)
f1_score(pred['ice_jam'], pred['ice_jam_pred'])

100%|████████████████████████████████████████████████████████████████████████████████| 258/258 [00:06<00:00, 40.58it/s]


In [86]:
train_model(train_df)

100%|████████████████████████████████████████████████████████████████████████████████| 258/258 [00:08<00:00, 30.87it/s]


In [17]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(rfc, f)